In [1]:
import torch as nn
import numpy as np
import pandas as pd

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

class SingleScoreCNN(nn.Module):
    def __init__(self):
        super(SingleScoreCNN, self).__init__()
        # Define the layers of the CNN
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 64 * 8)  # Adjust the size based on the input size
        self.fc2 = nn.Linear(64 * 8, 64 * 2)  
        self.fc3 = nn.Linear(64 * 2, 64) 
        self.fc4 = nn.Linear(64, 1)
    
    def forward(self, x):
        # Define the forward pass
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = F.tanh(self.fc3(x))
        x = self.fc4(x)
        return x

# Example usage:
model = SingleScoreCNN()
output = model(torch.randn(1, 3, 64, 64))  # Example input tensor (batch_size, channels, height, width)

# Define the loss and optimizer
criterion = nn.MSELoss()  # For example, if it's a regression task
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [3]:
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class MeasuresDataset(Dataset):
    def __init__(self, csv_file, image_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        page = str(int(self.data.iloc[idx]['Page']))
        label = self.data.iloc[idx]['Final Score']
        
        img_path = os.path.join(self.image_dir, f"{page}.jpg")
        # print(page, label, img_path)
        image = Image.open(img_path).convert('RGB')
        
        # Apply transformations, if provided
        if self.transform:
            image = self.transform(image)
        
        return image, torch.tensor(label, dtype=torch.float)


transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize to the size expected by the network
    transforms.ToTensor()
])

dataset = MeasuresDataset(csv_file='measures.csv', image_dir='./combined', transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Iterate over the dataloader
# for images, labels in dataloader:
#     # Feed data to the model
#     output = model(images)
#     # Compute the loss, etc.
#     print(output, labels)

/Applications/Anaconda/anaconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Applications/Anaconda/anaconda3/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <9096C4C6-0664-3775-B06F-54E79C6B0E3F> /Applications/Anaconda/anaconda3/lib/python3.9/site-packages/torchvision/image.so
  Expected in:     <E459C462-F863-3A5A-AC9F-FD77B14BE845> /Applications/Anaconda/anaconda3/lib/python3.9/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [4]:
import torch
import torch.optim as optim
import torch.nn as nn

# Prepare model, loss function, and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for images, labels in dataloader:
        # Move data to GPU if available
        # images, labels = images.cuda(), labels.cuda() if torch.cuda.is_available() else (images, labels)
        
        optimizer.zero_grad()  # Zero the parameter gradients
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs.squeeze(), labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        # Accumulate the loss
        epoch_loss += loss.item()
    
    # Compute and print the average loss for the epoch
    avg_loss = epoch_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], MSE Loss: {avg_loss:.4f}")

/Applications/Anaconda/anaconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/20], MSE Loss: 0.4394
Epoch [2/20], MSE Loss: 0.3991
Epoch [3/20], MSE Loss: 0.3691
Epoch [4/20], MSE Loss: 0.3849
Epoch [5/20], MSE Loss: 0.3790
Epoch [6/20], MSE Loss: 0.3698
Epoch [7/20], MSE Loss: 0.3704
Epoch [8/20], MSE Loss: 0.3664
Epoch [9/20], MSE Loss: 0.3523
Epoch [10/20], MSE Loss: 0.3464
Epoch [11/20], MSE Loss: 0.3373
Epoch [12/20], MSE Loss: 0.3465
Epoch [13/20], MSE Loss: 0.3588
Epoch [14/20], MSE Loss: 0.3285
Epoch [15/20], MSE Loss: 0.3150
Epoch [16/20], MSE Loss: 0.2762
Epoch [17/20], MSE Loss: 0.2285
Epoch [18/20], MSE Loss: 0.2542
Epoch [19/20], MSE Loss: 0.1392
Epoch [20/20], MSE Loss: 0.0857
